#### 9-3-1. k-NN 알고리즘
 - 지도학습, 분류(classify)에 해당하는 알고리즘.
 - 매칭 함수인 knnMatch() 함수를 내부적으로 사용함.
 - OpenCV에서는 cv2.ml.StatModel 추상 클래스를 상속받아 구현함.

 - 학습 데이터가 어느 부류에 해당하는지 알고 있을 때, 새로운 데이터를 주면 이것이 어느 부류에 해당하는지 예측함.
 - 예측해야 하는 새로운 점과 가장 가까운 이웃이 어느 부류로 분류되는지에 따라 새로운 점을 분류하는 방식.
 - 'K값'을 조정하여 '가장 가까운 이웃의 범위'를 조정할 수 있음.

 - A, B의 2가지의 데이터가 분포되어 있는 가운데 새로운 데이터가 들어왔다고 가정해보자.
 - 이때 k-NN 알고리즘은 새로운 데이터를 그 데이터가 위치한 곳으로부터 가장 가까이에 있는 데이터와 같은 종류로 분류한다. 
 - 그러나 어떤 범위 내에 A가 더 많이 들어있는지, B가 더 많이 들어있는지는 K값을 어떻게 조정하느냐에 따라 달라진다. 결국 K값에 따라 새로운 데이터 분류 결과도 달라진다.

 - knn = cv2.ml.KNearest_create(): k-NN 알고리즘 객체 생성

        - retval, results, neighborResponses, dist = knn.findNearest(samples, k): 예측
            * samples: 입력 데이터
            * k: 이웃 범위 지정을 위한 K값(단, K > 1)
            * results: 입력 데이터에 대한 예측 결과. 입력 데이터와 같은 크기의 배열.
            * neighborResponses: K 범위 내에 있는 이웃 데이터
            * dist: 입력 데이터와 이웃 데이터와의 거리
            * retval: 예측 결과 데이터. 입력 데이터가 1개인 경우.

In [ ]:
#practice. k-NN 난수 분류
import numpy as np
import matplotlib.pyplot as plt
import cv2

#0~99 사이의 랜덤한 25*2개 데이터 생성 -- ①
trainData = np.random.randint(0, 100, (25, 2)).astype(np.float32)

#0~1 사이의 랜덤한 수 25*1개 레이블 생성 -- ②
labels = np.random.randint(0, 2, (25, 1))

#레이블 값 0과 같은 자리는 red, 1과 같은 자리는 blue로 분류해서 표시 -- ③
red = trainData[labels.ravel() == 0]
blue = trainData[labels.ravel() == 1]

plt.scatter(red[:, 0], red[:, 1], 80, 'r', '^')   #붉은 삼각형
plt.scatter(blue[:, 0], blue[:, 1], 80, 'b', 's') #푸른 사각형

#k-NN 알고리즘으로 분류할, 0~99 사이의 랜덤 수 신규 데이터 생성 -- ④
newcomer = np.random.randint(0, 100, (1, 2)).astype(np.float32)
plt.scatter(newcomer[:, 0], newcomer[:, 1], 80, 'g', 'o')  #초록색 원

#KNearest 알고리즘 객체 생성 -- ⑤
knn = cv2.ml.KNearest_create()

#train, 행 단위 샘플 -- ⑥
knn.train(trainData, cv2.ml.ROW_SAMPLE, labels) #train data와 레이블 학습 #cv2.ml.ROW_SAMPLE: 학습 데이터가 행 단위임을 알림.

#예측 -- ⑦
#ret, results = knn.predict(newcomer)보다는, 예측에 사용한 최근접 이웃에 대한 정보를 추가로 제공하는 findNearest 함수가 더 나음.
ret, results, neighbors, dist = knn.findNearest(newcomer, 3) #K = 3
#ret = 예측 결과. #neighbors = K 범위 내에 있던 학습 데이터 #dist = 각 학습 데이터와의 거리

#결과 출력
print('ret: %s, result: %s, neighbors: %s, distance: %s' %(ret, results, neighbors, dist))
plt.annotate('red' if ret == 0.0 else 'blue', xy = newcomer[0], xytext = (newcomer[0]+1))
plt.show()

In [ ]:
#practice. k-NN 영화 장르 분류
#영화 장면에서 발차기 장면과 키스 장면의 횟수로 액션물/멜로물 장르 분류.
import cv2
import numpy as np
import matplotlib.pyplot as plt

#0~99 사이의 랜덤 값(25*2) -- ①
trainData = np.random.randint(0, 100, (25, 2)).astype(np.float32)
#trainData[0]: Kick, trainData[1]: kiss, kick > kiss ? 1:0 -- ②    #kick > kiss인 영화는 1(액션물), 그렇지 않은 것으 0(멜로물)
responses = (trainData[:, 0] > trainData[:, 1]).astype(np.float32)
#0: action, 1: romantic -- ③   #훈련데이터 분류
action = trainData[responses == 0]
romantic = trainData[responses == 1]

#action은 파란색 삼각형, romantic은 빨간색 동그라미로 표시 -- ④
plt.scatter(action[:, 0], action[:, 1], 80, 'b', '^', label = 'action')
plt.scatter(romantic[:, 1], romantic[:, 1], 80, 'r', 'o', label = 'romantic')

#새로운 데이터 생성, 0~99 랜덤 수 1*2, 초록색 사각형으로 표시 -- ⑤
newcomer = np.random.randint(0, 100, (1, 2)).astype(np.float32)
plt.scatter(newcomer[:, 0], newcomer[:, 1], 200, 'g', 's', label = 'new')

#KNearest 알고리즘 생성 및 훈련 -- ⑥
knn = cv2.ml.KNearest_create()
knn.train(trainData, cv2.ml.ROW_SAMPLE, responses)

#결과 예측 -- ⑦
ret, results, neighbors, dist = knn.findNearest(newcomer, 3) #K = 3
print("ret: %s, results: %s, neighbors: %s, dist: %s" %(ret, results, neighbors, dist))

#새로운 결과에 화살표로 표시
anno_x, anno_y = newcomer.ravel()
label = 'action' if results == 0 else 'romantic'
plt.annotate(label, xy = (anno_x + 1, anno_y + 1), xytext = (anno_x + 5, anno_y + 10), arrowprops = {'color':'black'})
plt.xlabel('kiss'); plt.ylabel('kick')
plt.legend(loc = 'upper right')
plt.show()

#### 9-3-2. 손글씨 인식
 - 아래 모듈은 mnist 데이터를 훈련 데이터용으로 분류한 모듈임.

In [ ]:
#practice. knn으로 MNIST 손글씨 숫자 학습
import numpy as np
import cv2
import mnist    #위 모듈을 mnist.py 의 별도의 python 파일로 저장한 후 불러올 경우 import 할 것.

#훈련 데이터, 검증 데이터 가져오기 -- ①
train, train_labels = mnist.getTrain()   #mnist.py라는 별도의 모듈이 있을 경우 이 코드를 사용.
test, test_labels = mnist.getTest()      #mnist.py라는 별도의 모듈이 있을 경우 이 코드를 사용.

#knn 객체 생성 및 훈련 -- ②
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
#k값을 1 ~ 10까지 변경하면서 예측 -- ③    #k값에 따른 정확도 변화를 확인하기 위함.
for k in range(1, 11):
    #결과 예측 -- ④
    ret, result, neighbors, distance = knn.findNearest(test, k=k) #result => test와 같이 500개
    #정확도 계산 및 출력 -- ⑤
    correct = np.sum(result == test_labels) # result와 test_labels 비교하여, 1에 해당하는 True값들 합함.
    accuracy = correct / result.size * 100.0 # 정답 개수 ÷ 전체 개수 * 100 = 정확도
    print("K:%d, Accuracy: %.2f%%(%d/%d)" %(k, accuracy, correct, result.size))

In [ ]:
#practice. 손글씨 숫자 인식
import numpy as np
import cv2
import mnist    #mnist.py 로 모듈이 별도의 python 파일로 있을 경우 import 할 것.

#훈련 데이터 가져오기 -- ①
train, train_labels = mnist.getData()   #mnist.py라는 별도의 모듈이 있을 경우 이 코드를 사용.

#knn 객체 생성 및 학습 -- ②
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)

#인식시킬 손글씨 이미지 읽기 -- ③
image = cv2.imread('../img/4027.png')
cv2.imshow('image', image)
cv2.waitKey(0)

#그레이스케일 변환과 스레시홀드 -- ④
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
_, gray = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

#최외곽 컨투어만 찾기 -- ⑤
contours, img = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
#OpenCV 4부터는 cv2.findContours()[0]이 contour


#모든 컨투어 순회 -- ⑥
for c in contours:
    #컨투어를 감싸는 외접 사각형으로 숫자 영역 좌표 구하기 -- ⑦
    x, y, w, h = cv2.boundingRect(c)
    #외접사각형의 크기는 너무 작은 것은 제외 -- ⑧
    if w >= 5 and h >= 25:
        #숫자 영역만 roi로 확보하고 사각형 그리기 -- ⑨
        roi = gray[y: y + h, x: x + w]
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #테스트 데이터 형식으로 변환 -- ⑩
        data = mnist.digit2data(roi)
        #data = mnist.digit2data(roi)   #mnist.py라는 별도의 모듈이 있을 경우 이 코드를 사용.
        #결과를 예측해서 이미지에 표시 -- ⑪
        ret, result, neighbors, dist = knn.findNearest(data, k=1)
        cv2.putText(image, "%d"%ret, (x, y + 155), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)
        cv2.imshow("image", image)
        cv2.waitKey(0)
cv2.destroyAllWindows()